In [ ]:
from numpy import *

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],       # 첫번째 글의 제목 / 단어별로 잘 정돈되어 다 쪼개져 있음.
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],               
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],  
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not                                     
    return postingList,classVec                                  


def createVocabList(dataSet):
    vocabSet = set([])  #create empty set        # 비어있는 set을 만듬
    for document in dataSet:              # 데이터 셋에서 문서로 
        vocabSet = vocabSet | set(document) #union of the two sets    문서를 셋트로 해서 그 안의 단어들이 별개로 나옴
    return list(vocabSet)  # or   합집합( 중복 값은 없어짐 )을 해라.   => 둘다 집합이 set 이 되니까 가능함. 
   # => 6개의 문장에 쓰인 단어들을 뽑아낼 수 있음.

    
    
    
    
def setOfWords2Vec(vocabList, inputSet):                 # ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'] 들어오면.
    returnVec = [0]*len(vocabList)                          # 리스트가 0으로 다 바뀜.   
    for word in inputSet:                                          
        if word in vocabList:                           # 단어리스트의 단어를 하나씩 꺼내서
            returnVec[vocabList.index(word)] = 1                  # my : 31번째 위치에 1을 채움. /  dog : 24번째
        else: print "the word: %s is not in my Vocabulary!" % word
    return returnVec








def trainNB0(trainMatrix,trainCategory):              # setOfWords2Vec의 결과값이 들어감 
    numTrainDocs = len(trainMatrix)                   
    numWords = len(trainMatrix[0])                     # 7개 / 
    pAbusive = sum(trainCategory)/float(numTrainDocs)  # 단어 범주가 의 확률      classVec = [0,1,0,1,0,1]  
                #                # 
      #(인자)  p1, p2   
    p0Num = zeros(numWords); p1Num = zeros(numWords)      #change to ones()    pAbusive일 때의 
    p0Denom = 0.0; p1Denom = 0.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])         
    p1Vect = (p1Num/p1Denom)          #change to log()
    p0Vect = (p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive





def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
    
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)

def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 
    
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = range(50); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print "classification error",docList[docIndex]
    print 'the error rate is: ',float(errorCount)/len(testSet)
    #return vocabList,fullText

def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq = sorted(freqDict.iteritems(), key=operator.itemgetter(1), reverse=True) 
    return sortedFreq[:30]       

def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet=[]           #create test set
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print 'the error rate is: ',float(errorCount)/len(testSet)
    return vocabList,p0V,p1V

def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print "SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**"
    for item in sortedSF:
        print item[0]
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print "NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**"
    for item in sortedNY:
        print item[0]



In [1]:
#ch.04 Naive Bayes

In [2]:
#### 4.5.1 

In [11]:
import bayes

In [12]:
listOPosts, listClasses = bayes.loadDataSet()

In [13]:
listOPosts

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [14]:
listClasses

[0, 1, 0, 1, 0, 1]

In [18]:
myVocabList = bayes.createVocabList(listOPosts)
         # 단어가 나옴.

In [19]:
myVocabList, len(myVocabList)

(['cute',
  'love',
  'help',
  'garbage',
  'quit',
  'I',
  'problems',
  'is',
  'park',
  'stop',
  'flea',
  'dalmation',
  'licks',
  'food',
  'not',
  'him',
  'buying',
  'posting',
  'has',
  'worthless',
  'ate',
  'to',
  'maybe',
  'please',
  'dog',
  'how',
  'stupid',
  'so',
  'take',
  'mr',
  'steak',
  'my'],
 32)

In [22]:
bayes.setOfWords2Vec(myVocabList, listOPosts[0])
 # myVocabList 의 단어가 안나왔다 = 0
 #                     가 나왔다. = 1           => 더미테이블로 만들 수 있음.               

[0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [23]:
bayes.setOfWords2Vec(myVocabList, listOPosts[1])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0]

In [24]:
###

myVocabList

['cute',
 'love',
 'help',
 'garbage',
 'quit',
 'I',
 'problems',
 'is',
 'park',
 'stop',
 'flea',
 'dalmation',
 'licks',
 'food',
 'not',
 'him',
 'buying',
 'posting',
 'has',
 'worthless',
 'ate',
 'to',
 'maybe',
 'please',
 'dog',
 'how',
 'stupid',
 'so',
 'take',
 'mr',
 'steak',
 'my']

In [25]:
0 * len(myVocabList)

0

In [26]:
[0] * len(myVocabList)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [28]:
myVocabList.index('my')      # 마이의 위치(index 값)

31

In [29]:
myVocabList.index('dog')   

24

In [31]:
### 4.5.2

In [53]:
from numpy import *

In [54]:
import imp
imp.reload(bayes)

<module 'bayes' from 'bayes.py'>

In [55]:
listOposts, listClasses = bayes.loadDataSet()

In [56]:
myVocabList = bayes.createVocabList(listOPosts) # 어휨 목록을 만들고

In [57]:
trainMat = [] # 리스트를 만들겠다.

In [58]:
for postInDoc in listOPosts:
    trainMat.append(bayes.setOfWords2Vec(myVocabList, postInDoc))

In [59]:
len(trainMat)

6

In [60]:
trainMat[0][:10]
trainMat[:][:10]

[[0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0],
 [1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1],
 [0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1],
 [0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0]]

In [61]:
p0V, p1V, pAb = bayes.trainNB0(trainMat, listClasses)

In [62]:
pAb

0.5

In [63]:
p0V          # 이 전체에서 그 것이 나올 확률

array([0.04166667, 0.04166667, 0.04166667, 0.        , 0.        ,
       0.04166667, 0.04166667, 0.04166667, 0.        , 0.04166667,
       0.04166667, 0.04166667, 0.04166667, 0.        , 0.        ,
       0.08333333, 0.        , 0.        , 0.04166667, 0.        ,
       0.04166667, 0.04166667, 0.        , 0.04166667, 0.04166667,
       0.04166667, 0.        , 0.04166667, 0.        , 0.04166667,
       0.04166667, 0.125     ])

In [64]:
p1V

array([0.        , 0.        , 0.        , 0.05263158, 0.05263158,
       0.        , 0.        , 0.        , 0.05263158, 0.05263158,
       0.        , 0.        , 0.        , 0.05263158, 0.05263158,
       0.05263158, 0.05263158, 0.05263158, 0.        , 0.10526316,
       0.        , 0.05263158, 0.05263158, 0.        , 0.10526316,
       0.        , 0.15789474, 0.        , 0.05263158, 0.        ,
       0.        , 0.        ])